In [3]:
pip install torch torchvision

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.9/63.9 MB 15.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 9.6 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 10.7 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: sympy
    Found existing installation: sympy 1.12
    Uninstalling sympy-1.12:
      Successfully uninstalled sympy-1.12
Note: you may need to restart the kernel to use updated packages.


In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

class StockPredictionModel(nn.Module):
    def __init__(self, input_features=70, dropout_rate=0.1):
        super(StockPredictionModel, self).__init__()
        
        # 1D CNN layers with specified number of kernels (32, 64, 128)
        self.conv1 = nn.Conv1d(in_channels=input_features, out_channels=32, kernel_size=5, padding='same')
        self.conv2 = nn.Conv1d(in_channels=32, out_channels=64, kernel_size=5, padding='same')
        self.conv3 = nn.Conv1d(in_channels=64, out_channels=128, kernel_size=5, padding='same')
        
        # Activation function
        self.leaky_relu = nn.LeakyReLU()
        
        # Dropout layer
        self.dropout = nn.Dropout(dropout_rate)
        
        # Fully connected layers
        self.fc1 = nn.Linear(128, 220)
        self.fc2 = nn.Linear(220, 1)  # Final output for prediction
        
        # Batch normalization layers
        self.bn1 = nn.BatchNorm1d(32)
        self.bn2 = nn.BatchNorm1d(64)
        self.bn3 = nn.BatchNorm1d(128)
    
    def forward(self, x):
        # Reshape input for 1D convolution if needed
        # Input shape: (batch_size, sequence_length, input_features)
        # Required shape: (batch_size, input_features, sequence_length)
        x = x.transpose(1, 2)
        
        # Apply CNN layers
        x = self.leaky_relu(self.bn1(self.conv1(x)))
        x = self.dropout(x)
        
        x = self.leaky_relu(self.bn2(self.conv2(x)))
        x = self.dropout(x)
        
        x = self.leaky_relu(self.bn3(self.conv3(x)))
        x = self.dropout(x)
        
        # Global average pooling
        x = torch.mean(x, dim=2)
        
        # Fully connected layers
        x = self.leaky_relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        
        return x

# Training function
def train_model(model, train_loader, val_loader, num_epochs=200, learning_rate=0.001):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    
    for epoch in range(num_epochs):
        model.train()
        train_loss = 0
        for batch_idx, (data, target) in enumerate(train_loader):
            data, target = data.to(device), target.to(device)
            
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
        
        # Validation
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for data, target in val_loader:
                data, target = data.to(device), target.to(device)
                output = model(data)
                val_loss += criterion(output, target).item()
        
        if (epoch + 1) % 10 == 0:
            print(f'Epoch: {epoch+1}/{num_epochs}')
            print(f'Training Loss: {train_loss/len(train_loader):.6f}')
            print(f'Validation Loss: {val_loss/len(val_loader):.6f}\n')

# Example usage
def prepare_data(data, sequence_length):
    """
    Prepare data for the model
    data: numpy array of shape (n_samples, n_features)
    sequence_length: number of time steps to look back
    Returns:
        X: tensor of shape (n_samples, sequence_length, n_features)
        y: tensor of shape (n_samples, 1)
    """
    X, y = [], []
    for i in range(len(data) - sequence_length):
        X.append(data[i:(i + sequence_length), :])  # Keep all features
        y.append(data[i + sequence_length, 0])  # Assuming first feature is the target
    return torch.FloatTensor(np.array(X)), torch.FloatTensor(np.array(y)).reshape(-1, 1)

# Example of how to use the model
if __name__ == "__main__":
    # Initialize model with the specified parameters
    model = StockPredictionModel(input_features=70, dropout_rate=0.1)
    
    # Create dummy data for demonstration
    batch_size = 64
    sequence_length = 10
    n_features = 70
    n_samples = 1000
    
    # Generate dummy data
    dummy_data = np.random.randn(n_samples, n_features)
    X, y = prepare_data(dummy_data, sequence_length)
    
    # Create DataLoader
    dataset = torch.utils.data.TensorDataset(X, y)
    train_size = int(0.8 * len(dataset))
    val_size = len(dataset) - train_size
    train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])
    
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size)
    
    # Train the model
    train_model(model, train_loader, val_loader)

Epoch: 10/200
Training Loss: 0.516214
Validation Loss: 1.240808

Epoch: 20/200
Training Loss: 0.223531
Validation Loss: 1.464521

Epoch: 30/200
Training Loss: 0.154579
Validation Loss: 1.536503

Epoch: 40/200
Training Loss: 0.110389
Validation Loss: 1.524194

Epoch: 50/200
Training Loss: 0.090701
Validation Loss: 1.506324

Epoch: 60/200
Training Loss: 0.094286
Validation Loss: 1.387596

Epoch: 70/200
Training Loss: 0.077886
Validation Loss: 1.541332

Epoch: 80/200
Training Loss: 0.103308
Validation Loss: 1.451421

Epoch: 90/200
Training Loss: 0.056926
Validation Loss: 1.524563

Epoch: 100/200
Training Loss: 0.061286
Validation Loss: 1.506006

Epoch: 110/200
Training Loss: 0.052225
Validation Loss: 1.461072

Epoch: 120/200
Training Loss: 0.050127
Validation Loss: 1.380082

Epoch: 130/200
Training Loss: 0.042810
Validation Loss: 1.415316

Epoch: 140/200
Training Loss: 0.055436
Validation Loss: 1.370927

Epoch: 150/200
Training Loss: 0.043712
Validation Loss: 1.424672

Epoch: 160/200
Trai